# Imports

In [ ]:
import pandas as pd
import pyodbc
import numpy as np

# Database connection credentials

In [8]:
server = 'bixsql01.bixpert.local'  
database = 'JDS_Verseny_DEV'
username = 'sa'
password = 'Galeria1'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

## Event table

In [9]:
query = "SELECT  * FROM L_EVENT"

events = pd.read_sql(query, conn)

events.head(100)

/var/folders/h8/jsx_20090y338vjhdy9lwbnc0000gn/T/ipykernel_4253/3097665812.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events = pd.read_sql(query, conn)


,EVENTID,EVENTDESC,EVENTLONGDESC,EVENT_SORTID,EVENT_VALID
0,1,Not_Defined,None,1,1
1,2,$ae_first_open,None,1,1
2,3,$ae_iap,None,1,1
3,4,$ae_session,None,1,1
4,5,$ae_updated,None,1,1
...,...,...,...,...,...
95,96,CaptureService.setCustomExposure,None,1,1
96,97,checkSubscriptionStatus.Error,None,1,1
97,98,ChooseAccountViewController<ChooseFacebookAcco...,None,1,1
98,99,ChooseAccountViewController<ChooseFacebookAcco...,None,1,1


# Dataset

In [11]:
query = "select * from teleprompter_hashed where event in ('trial_started_event', 'trial_cancelled_event', 'trial_converted_event')"

first_df = pd.read_sql(query, conn)

first_df.head(100)

/var/folders/h8/jsx_20090y338vjhdy9lwbnc0000gn/T/ipykernel_4253/4198356449.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  first_df = pd.read_sql(query, conn)


,event,event_time,distinct_id,os_version,country_code
0,trial_started_event,1739723957,$device:user_06953970413949,unknown,unknown
1,trial_converted_event,1737753123,$device:user_70704306992540,unknown,unknown
2,trial_started_event,1739423863,user_43976219804144,unknown,unknown
3,trial_cancelled_event,1739772250,user_47696106159422,unknown,unknown
4,trial_started_event,1737624967,$device:user_62726075833536,unknown,unknown
...,...,...,...,...,...
95,trial_converted_event,1739842750,$device:user_11681579362468,unknown,unknown
96,trial_started_event,1736892060,$device:user_02494621415355,unknown,unknown
97,trial_started_event,1736071175,$device:user_65449298927066,unknown,unknown
98,trial_started_event,1736023838,$device:user_27476709039245,unknown,unknown


## Hány user indított trailt?

In [22]:
start_trial_users = first_df.loc[first_df.event=='trial_started_event', 'distinct_id'].unique()
print(f"A megadott idoszakban {len(start_trial_users)} user kezdte meg a tesztidoszakot.")

A megadott idoszakban 19591 user kezdte meg a tesztidoszakot.


## Ebből hány user konvertált?

In [34]:
trial_converted_users = first_df.loc[first_df.event=='trial_converted_event', 'distinct_id'].unique()

number_of_converted = len(np.intersect1d(start_trial_users, trial_converted_users))

print(f"A megadott idoszakban {number_of_converted} user konvertalt azok kozul, akik a megadott idoszakban kezdtek meg a tesztidoszakot.")

A megadott idoszakban 3384 user konvertalt azok kozul, akik a megadott idoszakban kezdtek meg a tesztidoszakot.


## Ebből hány mondta le az előfizetést?
##### Itt arra vagyunk kiváncsiak, hogy hány user indított tesztidőszakot, fizetett elő és mondta le az előfizetést.

In [38]:
trial_cancelled_users = first_df.loc[first_df.event=='trial_cancelled_event', 'distinct_id'].unique()

number_of_cancelled = len(np.intersect1d(trial_converted_users, trial_cancelled_users))

print(f"{number_of_cancelled} user kezdte meg a tesztidoszakot, akik utana konvertaltak is es le is mondtak az elofizetest a rendelkezesre allo idoszakban (2025.01.01 – 2025.02.28.).")

897 user kezdte meg a tesztidoszakot, akik utana konvertaltak is es le is mondtak az elofizetest a rendelkezesre allo idoszakban (2025.01.01 – 2025.02.28.).
